# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Diana Fernanda Barbosa Dueñas  |
| **Fecha**      | 16/02/2026  |
| **Expediente** | 756920  |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

**Train** es justamente para entrenar a nuestro modelo y que nos pueda dar un resultado deseado

**Test** es para analizar como es que va nuestro modelo con las muestras y poder corregir o mejorar.

**Split** es un pequño conjunto de nuestra población por lo que nos ayuda a poder generalizar nuestros datos y que al final no sean tantas muestras las cuales analizar.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

No, ya que las muestras podrian ser bastantes por lo que no nos podría dar un resultado real, por el hecho de que esta bastante cargado, entonces esto hace que nuestro modelo sea un poco más "irreal". También porque dependiendo de la muestra puede influir más una que otra por eso generalizamos.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Motor Trend Car Road Tests.xlsx to Motor Trend Car Road Tests.xlsx


In [6]:
import pandas as pd
Motor = pd.read_excel('Motor Trend Car Road Tests.xlsx')
Motor.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [7]:
# Eliminar columna
Motor = Motor.drop(columns=['model'])

In [12]:
# Variables
X = Motor.drop(columns=['mpg'])
y = Motor['mpg']

In [15]:
# Muestras de data
n = len(Motor)

# Guardar errores
mse_lista = []

## **Leave-One-Out Cross Validation**

In [18]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [23]:
for i in range(n):

    X_test = X.iloc[i].values.reshape(1, -1)
    y_test = np.array(y.iloc[i]).reshape(1,)

    X_train = np.delete(X.values, i, axis=0)
    y_train = np.delete(y.values, i, axis=0)

    pred = modelo.predict(X_test)

    mse = mean_squared_error(y_test, pred)
    mse_lista.append(mse)

In [21]:
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [22]:
# Resultados
print("MSE promedio:", np.mean(mse_lista))
print("Desviación estándar:", np.std(mse_lista))

MSE promedio: 12.181558006901943
Desviación estándar: 17.067399871888522


Interpreta.

Primero eliminamos la columna como se pide, y usamos "mpg" como nuestra "y". Por lo que así empecé a usar las demás columnas como mis muestras para poder entrenar mi modelo. Después de entrenarlo, mi modelo intenta predecir el valor real de esa fila que dejamos fuera. Luego comparó la predicción con el valor verdadero y calculó el error usando la media y la desviación estandar para ver qué tan cerca estuvo el modelo. Ese error lo guardamos en una lista. Repetimos este proceso con cada fila del dataset para asegurarnos de probar el modelo con todos los datos posibles y al final poder analizar qué tan bien predice en general.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3